In [1]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, sys
sys.path.insert(0, "/Users/lili/dropbox_lili/common_functions/outlier_analysis/")
import blackSheepCPTACmodule as ol

In [2]:
import importlib
importlib.reload(ol)

<module 'blackSheepCPTACmodule' from 'C:\\Users\\Daniel\\Documents\\GitHub\\WhenMutationsMatter\\Daniel\\outlier_analysis\\blackSheepCPTACmodule.py'>

In [3]:
import cptac.endometrial as en

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.1
******
You have loaded the cptac endometrial dataset. To view available
dataframes, use cptac.endometrial.list_data(). To view available
functions for accessing and manipulating the dataframes, use
cptac.endometrial.list_api().
endometrial data version: 2.1

Loading Dictionary...
Loading cptac endometrial data:
Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available

In [4]:
%%timeit

proteomics = en.get_proteomics()
transcriptomics = en.get_transcriptomics()
small_proteomics = proteomics.iloc[0:100, :]
outliers = ol.make_outliers_table(transcriptomics, iqrs=2.0, 
                                  up_or_down='up', aggregate=False, 
                                  frac_table=True)

9.07 s ± 3.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
clinical = en.get_clinical()
annotations = clinical[['Histologic_Grade_FIGO','Diabetes', 'Race', 'Age']]
set(clinical)

{'Age',
 'BMI',
 'Clin_Stage_Dist_Mets-cM',
 'Country',
 'Diabetes',
 'Ethnicity',
 'FIGO_stage',
 'Gender',
 'Histologic_Grade_FIGO',
 'Histologic_type',
 'LVSI',
 'Myometrial_invasion_Specify',
 'Num_full_term_pregnancies',
 'Path_Stage_Dist_Mets-pM',
 'Path_Stage_Primary_Tumor-pT',
 'Path_Stage_Reg_Lymph_Nodes-pN',
 'Patient_ID',
 'Proteomics_Tumor_Normal',
 'Race',
 'Treatment_naive',
 'Tumor_Focality',
 'Tumor_Site',
 'Tumor_Site_Other',
 'Tumor_Size_cm',
 'Tumor_purity',
 'tumor_Stage-Pathological'}

In [ ]:
##Create a function that binarizes classifiers
##Need to make this more resilient and create a function
##that allows you to filter the end enrichment table by p-values < 0.05
def binarize(df, boundary_name, original_qualifier, boundary):
    annotations[boundary_name] = annotations[original_qualifier] > boundary
    print(annotations)
    
#binarize(annotations, 'Over_50', 'Age', 50)

In [14]:
annotations.describe()

,Age
count,95.000000
mean,63.473684
std,10.109867
min,38.000000
25%,58.000000
50%,64.000000
75%,69.000000
max,90.000000


In [47]:
def returnSubsetDF(df):
    print('Here are your options')
    print(set(df))
    boundary_names = []
    print('Please input the columns you would like to binarize. Type "done" to finish.')
    boundary = ''
    while boundary != 'done':
        boundary = input()
        if boundary == 'done':
            break
        elif boundary not in set(df):
            print('Invalid input\nHere are your options:')
            print(set(df))
        else:
            boundary_names.append(boundary)
    new_df = pd.DataFrame()
    for boundary in boundary_names:
        new_df[boundary] = df[boundary]
    return(new_df)

smaller_df = returnSubsetDF(clinical)

Here are your options
{'Age', 'Myometrial_invasion_Specify', 'BMI', 'Histologic_Grade_FIGO', 'Path_Stage_Reg_Lymph_Nodes-pN', 'Tumor_purity', 'Path_Stage_Dist_Mets-pM', 'Patient_ID', 'Diabetes', 'Num_full_term_pregnancies', 'LVSI', 'Tumor_Focality', 'Tumor_Site', 'Histologic_type', 'Clin_Stage_Dist_Mets-cM', 'tumor_Stage-Pathological', 'Ethnicity', 'Path_Stage_Primary_Tumor-pT', 'Proteomics_Tumor_Normal', 'FIGO_stage', 'Gender', 'Country', 'Treatment_naive', 'Tumor_Site_Other', 'Race', 'Tumor_Size_cm'}
Please input the columns you would like to binarize. Type "done" to finish.
FIGO_stage
Gender
Country
done


In [55]:
def binarizeV2(df):
    boundary_names = list(set(df))
    example_dict = {}
    print('Please provide a dictionary to binarize the following columns:\n')
    for boundary in boundary_names:
          print(boundary)
          example_dict[boundary] = {'option1': ['values', 'that', 'correspond', 'with', 'option1'],
                                   'option2': ['values', 'that', 'correspond', 'with', 'option2']}
    print('\nHere is an example of how that dictionary might be setup')
    print(example_dict)
binarizeV2(smaller_df)

Please provide a dictionary to binarize the following columns:

FIGO_stage
Gender
Country

Here is an example of how that dictionary might be setup
{'FIGO_stage': {'option1': ['values', 'that', 'correspond', 'with', 'option1'], 'option2': ['values', 'that', 'correspond', 'with', 'option2']}, 'Gender': {'option1': ['values', 'that', 'correspond', 'with', 'option1'], 'option2': ['values', 'that', 'correspond', 'with', 'option2']}, 'Country': {'option1': ['values', 'that', 'correspond', 'with', 'option1'], 'option2': ['values', 'that', 'correspond', 'with', 'option2']}}


In [6]:
set(clinical['Race'])

{nan, 'White', 'Not Reported', 'Asian', 'Black or African American'}

In [49]:
'''
for item in set(clinical):
    print(item)
    print(set(clinical[item]))
    print('\n')
'''

"\nfor item in set(clinical):\n    print(item)\n    print(set(clinical[item]))\n    print('\n')\n"

In [62]:
# Here is an example of binarizing
# Here are the columns I want to binarize out of the clinical dataset
# Keys: columns, Values: dictionary of options and values
''' Here is what a dictionary of options and values would look like
{'option1': ['values', 'that', 'correspond', 'with', 'option1'],
 'option2': ['values', 'that', 'correspond', 'with', 'option2']}


example_dictionary = {'Race': {'Asian': ['Asian'], 
                               'Not_Asian' : ['White', np.nan, 'Black or African American', 
                                              'Not Reported', ' White']}, 
                      'Diabetes' : {'Diabetic' : ['Yes'], 
                                    'Not_Diabetic': ['No', 'Unknown', np.nan]}
                     }


for key, value in example_dictionary.items():
    print(key)
    for k, v in value.items():
        print(k)
        print(v)

new_df = pd.DataFrame()
for key, value in example_dictionary.items():
    for k, v in value.items():
        for item in v:

def binarizeV3(df, dictionary):
    new_df = pd.DataFrame()
    for item in clinical.iterrows():
        for key, value in example_dictionary.items():
            print(key)
            for k, v in value.items():
                if item in v:
                    new_df[key] = k
                else:
                    new_df[key] = v
'''

'\nfor key, value in example_dictionary.items():\n    print(key)\n    for k, v in value.items():\n        print(k)\n        print(v)\n\nnew_df = pd.DataFrame()\nfor key, value in example_dictionary.items():\n    for k, v in value.items():\n        for item in v:\n\ndef binarizeV3(df, dictionary):\n    new_df = pd.DataFrame()\n    for item in clinical.iterrows():\n        for key, value in example_dictionary.items():\n            print(key)\n            for k, v in value.items():\n                if item in v:\n                    new_df[key] = k\n                else:\n                    new_df[key] = v\n'

In [41]:
# Version 4 Probably the best version so far, even though it only fully works for age
binary_columns = clinical[['Age', 'Race']]

def binarizeContinuous(my_list, cut_off, replacement_low, replacement_high):
    return [ replacement_low if (x < cut_off) else replacement_high if x >= cut_off else x for x in my_list ]

print(binarizeContinuous(binary_columns['Age'], 50 , 'Young', 'Old'))

['Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Young', 'Old', 'Old', 'Young', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Young', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Young', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Young', 'Old', 'Old', 'Old', 'Old', 'Old', 'Young', 'Old', 'Old', 'Young', 'Old', 'Young', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', 'Old', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [59]:
results = ol.compare_groups_outliers(outliers, annotations, frac_filter=.1)
results

No rows had outliers in at least 0.1 of Histologic_Grade_FIGO FIGO grade 2 samples
Testing 1 rows for enrichment in Histologic_Grade_FIGO FIGO grade 1 samples
No rows had outliers in at least 0.1 of Diabetes No samples
Testing 1 rows for enrichment in Diabetes Yes samples
No rows had outliers in at least 0.1 of Race White samples
Testing 1 rows for enrichment in Race Black or African American samples


,Histologic_Grade_FIGO_FIGO grade 1_enrichment_FDR,Diabetes_Yes_enrichment_FDR,Race_Black or African American_enrichment_FDR
A1BG,NaN,NaN,NaN
A2M,NaN,NaN,NaN
A2ML1,NaN,NaN,NaN
A4GALT,0.412557,0.401228,NaN
AAAS,NaN,NaN,NaN
AACS,NaN,NaN,NaN
AADAT,NaN,NaN,NaN
AAED1,NaN,NaN,0.113122
AAGAB,NaN,NaN,NaN
AAK1,NaN,NaN,NaN


In [25]:
binarizeV2(clinical)

Here are your options
{'Myometrial_invasion_Specify', 'Tumor_Site', 'Age', 'Path_Stage_Reg_Lymph_Nodes-pN', 'FIGO_stage', 'Num_full_term_pregnancies', 'tumor_Stage-Pathological', 'BMI', 'Patient_ID', 'Tumor_Focality', 'Path_Stage_Dist_Mets-pM', 'Histologic_Grade_FIGO', 'Proteomics_Tumor_Normal', 'Path_Stage_Primary_Tumor-pT', 'Diabetes', 'Race', 'Gender', 'Tumor_Size_cm', 'Treatment_naive', 'Country', 'Tumor_Site_Other', 'Ethnicity', 'Histologic_type', 'Clin_Stage_Dist_Mets-cM', 'LVSI', 'Tumor_purity'}
FIGO_stage
Please provide a dictionary to binarize FIGO_stage. Here is an example:
{'columnName1': {'option1': ['values', 'that', 'correspond', 'with', 'option1'], 'option2': ['values', 'that', 'correspond', 'with', 'option2']}, 'columnName2': {'option1': ['values', 'that', 'correspond', 'with', 'option1'], 'option2': ['values', 'that', 'correspond', 'with', 'option2']}}
